In [1]:
import sys
print(sys.executable)
print(sys.version)
import langchain
print(langchain.__version__)


c:\Users\Lenovo\OneDrive\Desktop\MedHelp-end-to-end\.venv\Scripts\python.exe
3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
0.3.21


In [2]:
# !pip install langchain langchain_community langchain_openai langchain_experimental pypdf sentence-transformers==2.2.2 flask pinecone[grpc] langchain-pinecone

In [3]:
print("ok")

ok


In [4]:
%pwd

'c:\\Users\\Lenovo\\OneDrive\\Desktop\\MedHelp-end-to-end\\research'

In [5]:
import os
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\Lenovo\\OneDrive\\Desktop\\MedHelp-end-to-end'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
#extracting Data from the pdf file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [9]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
# extracted_data

In [11]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text CHunks", len(text_chunks))

Length of Text CHunks 6970


In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [14]:
%pip install --upgrade sentence-transformers
import sentence_transformers
print(sentence_transformers.__version__)
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

Note: you may need to restart the kernel to use updated packages.


c:\Users\Lenovo\OneDrive\Desktop\MedHelp-end-to-end\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.4.1


In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_500\2162885500.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [16]:
query_result = embeddings.embed_query("Helloworld")
print("length", len(query_result))

length 384


In [17]:
%pip install --upgrade google-generativeai>=0.3.0 langchain-google-genai>=0.0.7
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')
print("GEMINI_API_KEY exists:", "GEMINI_API_KEY" in os.environ)
print("PINECONE_API_KEY exists:", "PINECONE_API_KEY" in os.environ)

Note: you may need to restart the kernel to use updated packages.
GEMINI_API_KEY exists: True
PINECONE_API_KEY exists: True


In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [19]:
# #Embeded each chunk and upset the embeddings into your Pine inddeex
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name=index_name,
#     embedding=embeddings,
# )

In [20]:
#Load Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [21]:
docsearch

In [22]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [23]:
retrived_docs = retriever.invoke("What is thyroid?")

In [24]:
retrived_docs

[Document(id='c1690a7c-e4bd-4101-b243-8912f120f27b', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 500.0, 'page_label': '501', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='local anesthesia.\nEndometrial cancer—Cancer of the inner mucous\nmembrane of the uterus.\nFibroids, or fibroid tumors—Fibroid tumors are\nnon-cancerous (benign) growths in the uterus.\nThey occur in 30–40% of women over age 40,\nand do not need to be removed unless they are\ncausing symptoms that interfere with a woman’s\nnormal activities.\nHypothyroidism—A disorder in which the thyroid\ngland produces too little thyroid hormone causing\na decrease in the rate of metabolism with associat-'),
 Document(id='093b7cc9-06c7-4478-b4c2-53f89ad68456', metadata={'author': '', 'creationdate': '2017-05-01T10

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import google.generativeai as genai

# Configure Gemini with correct API version
# genai.configure(
#     api_key=GEMINI_API_KEY,
#     transport="rest",  # Force REST API for latest version compatibility
#     client_options={"api_endpoint": "generativelanguage.googleapis.com/v1beta"}
# )

# # Verify available models
# print("Available Models:")
# for model in genai.list_models():
#     print(f"- {model.name}")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.4,
    max_output_tokens=500,
    google_api_key=GEMINI_API_KEY,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }
)

In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),  # Changed to "input"
])

In [28]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [51]:
# from langchain_core.runnables import RunnablePassthrough

# # Create a custom chain that expects "prompt" input
# rag_chain = RunnablePassthrough.assign(
#     context=lambda x: retriever.invoke(x["prompt"])
# ).assign(
#     answer=question_answer_chain
# )

In [29]:
try:
    response = rag_chain.invoke({"input": "What is Thyroid?"})
    print(response["answer"])
except Exception as e:
    print(f"Error: {str(e)}")

The thyroid gland is located at the base of the neck. Hypothyroidism is a disorder in which the thyroid gland produces too little thyroid hormone. This causes a decrease in the rate of metabolism.


In [39]:
response = rag_chain.invoke({"input" : "i am caffeine addict?"})
print(response["answer"])

Heavy caffeine use can lead to dependence, and people who use large amounts of caffeine over long periods build up a tolerance to it. If an individual stops using caffeine abruptly, withdrawal symptoms may occur. You should check with a pharmacist or physician to find out how much caffeine is safe for you.
